In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        #<span id="productTitle" class="a-size-large">
        # Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)
        # </span>


        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available



In [3]:

if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=cricket+bat&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


<ipython-input-3-5296b5e69838>:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [5]:
amazon_df

,title,price,rating,reviews,availability
1,Nerf Foam Cricket Set - Full Starter Cricket w...,,4.4 out of 5 stars,43 ratings,Not Available
2,SS Cricket Premium Kashmir Willow Leather Ball...,,3.9 out of 5 stars,143 ratings,Not Available
4,"SS Kashmir Willow Leather Ball Cricket Bat, Ex...",,4.3 out of 5 stars,40 ratings,Not Available
5,"Junior Cricket Bat Set Wooden Gift Size 4, 6 I...",,4.6 out of 5 stars,56 ratings,Not Available
6,SS Noir Premium Kashmir Willow Cricket bat for...,,4.3 out of 5 stars,8 ratings,Not Available
8,Cricket Bat Kashmir Willow Wooden Handmade Adu...,,2.4 out of 5 stars,2 ratings,In Stock
9,SS Sunridges SS Sky Super Premium Cricket Kash...,,4.6 out of 5 stars,2 ratings,Not Available
12,SS Cricket Premium English Willow Cricket Bat ...,,4.3 out of 5 stars,8 ratings,Only 6 left in stock - order soon.
14,SPORTAXIS Wooden Kashmir Willow Cricket Bat wi...,,4.0 out of 5 stars,84 ratings,Not Available
15,"SS Kashmir Willow Leather Ball Cricket Bat, Ex...",,3.7 out of 5 stars,154 ratings,Not Available


In [ ]:
# prompt: with amazon_df filter with availability="In Stock", sorty by rating and then by number of ratings

amazon_df = amazon_df[amazon_df['availability'] == 'In Stock']
amazon_df = amazon_df.sort_values(by=['rating', 'reviews'], ascending=False)
amazon_df
